## Function (aka Tool) that LLM can decide to use

In [37]:
# This is the function that we want the model to be able to call
import datetime

def get_delivery_date(order_id: str) -> datetime:
    print("get_delivery_date function called with order_id: ", order_id)
    return datetime.datetime.strptime("2023-12-31", "%Y-%m-%d")

## Defining LLM and tools and chat history

In [38]:
import openai
client = openai.Client()
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    }
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        },
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})
messages.append({"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"})
messages.append({"role": "user", "content": "i think it is order_12345"})


## Querying the LLM given the chat history and tools

In [39]:
response = client.chat.completions.create(
    model="gpt-4o", messages=messages, tools=tools
)
response

ChatCompletion(id='chatcmpl-AB4b0hiSepMtTGarEAgun3sUPuUiB', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kxEN0rC3Sasw7EcxodtT80Gt', function=Function(arguments='{"order_id":"order_12345"}', name='get_delivery_date'), type='function')]))], created=1727202638, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_e375328146', usage=CompletionUsage(completion_tokens=19, prompt_tokens=140, total_tokens=159, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

If the model decided that a function should be called, response.choices[0] should be like:

```python
Choice(
    finish_reason='tool_calls', 
    index=0, 
    logprobs=None, 
    message=chat.completionsMessage(
        content=None, 
        role='assistant', 
        function_call=None, 
        tool_calls=[
            chat.completionsMessageToolCall(
                id='call_62136354', 
                function=Function(
                    arguments='{"order_id":"order_12345"}', 
                    name='get_delivery_date'), 
                type='function')
        ])
)
```

## If LLM decided to use get_delivery_date tool, call it

In [40]:
# Extract the arguments for get_delivery_date
# Note this code assumes we have already determined that the model generated a function call. See below for a more production ready example that shows how to check if the model generated a function call
import json


tool_call = response.choices[0].message.tool_calls[0]
print('''*********Example tool_call:\n''', tool_call)
print('''*********Example tool_call.function.arguments:\n''', tool_call.function.arguments)
arguments = json.loads(tool_call.function.arguments)
order_id = arguments.get("order_id")
print('''*********Example order_id:\n''', order_id)

# Call the get_delivery_date function with the extracted order_id
if tool_call.function.name == "get_delivery_date":
    delivery_date = get_delivery_date(order_id)
print('''*********Example delivery_date:\n''', delivery_date)

*********Example tool_call:
 ChatCompletionMessageToolCall(id='call_kxEN0rC3Sasw7EcxodtT80Gt', function=Function(arguments='{"order_id":"order_12345"}', name='get_delivery_date'), type='function')
*********Example tool_call.function.arguments:
 {"order_id":"order_12345"}
*********Example order_id:
 order_12345
get_delivery_date function called with order_id:  order_12345
*********Example delivery_date:
 2023-12-31 00:00:00


## Send the function call result to LLM so it can paraphrase

In [41]:
# Create a message containing the result of the function call
function_call_result_message = {
    "role": "tool",
    "content": json.dumps(
        {
            "order_id": order_id,
            "delivery_date": delivery_date.strftime("%Y-%m-%d %H:%M:%S"),
        }
    ),
    "tool_call_id": response.choices[0].message.tool_calls[0].id,
}

# Prepare the chat completion call payload
messages_with_function_call = json.loads(json.dumps(messages))
messages_with_function_call.append(response.choices[0].message)
messages_with_function_call.append(function_call_result_message)
completion_payload = {
    "model": "gpt-4o",
    "messages": messages_with_function_call,
}

# Call the OpenAI API's chat completions endpoint to send the tool call result back to the model
response_with_function_call = openai.chat.completions.create(
    model=completion_payload["model"], messages=completion_payload["messages"]
)

# Print the response from the API. In this case it will typically contain a message such as "The delivery date for your order #12345 is xyz. Is there anything else I can help you with?"
print(response_with_function_call.choices[0].message.content)

Your order (ID: order_12345) is scheduled to be delivered on December 31, 2023. If you have any other questions or need further assistance, feel free to ask!
